# Multiple flows

1. Flow 1 runs, inputs bar1 and outputs bar2
2. Flow 2 runs, inputs Flow1.input, Flow1.output

In [2]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.WARNING)

import pandas as pd
from doltpy.core import Dolt
from doltpy.core.write import import_df

dolt = Dolt.init(".")

df_v1 = pd.DataFrame({"A": [1,1,1], "B": [1,1,1]})
df_v2 = pd.DataFrame({"A": [1,1,1,2,2,2], "B": [1,1,1,2,2,2]})

import_df(dolt, "bar", df_v1.reset_index(), ["index"], "create")
dolt.add(".")
dolt.add("bar")
dolt.commit("Initialize bar")

v1 = list(dolt.log(number="1").keys())[0]

import_df(dolt, "bar", df_v2.reset_index(), ["index"], "update")
dolt.add("bar")
dolt.commit("Add rows to bar")

v2 = list(dolt.log(number="1").keys())[0]

In [3]:
!poetry run python3 demo_one.py run

Metaflow 2.2.5.post33+gitc20afd4 executing MultiFlowDemo1 for user:max-hoffman
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2021-01-18 11:56:14.443 Workflow starting (run-id 1610999774434109):
2021-01-18 11:56:14.450 [1610999774434109/start/1 (pid 36833)] Task is starting.
2021-01-18 11:56:15.530 [1610999774434109/start/1 (pid 36833)] 01-18 11:56:15 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-18 11:56:15.564 [1610999774434109/start/1 (pid 36833)] 01-18 11:56:15 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-18 11:56:15.627 [1610999774434109/start/1 (pid 36833)] 01-18 11:56:15 doltpy.core.dolt INFO     * master                                        	m7t6n1cabcmghfqav3jsti66mpcqs4go
2021-01-18 11:56:15.788 [1610999774434109/start/1 (pid 36833)] 
2021-01-18 11:56:15.788 [1610999774434109/start/1 (pid 36833)] 01-18 11:56:15 dolt

2021-01-18 11:56:19.815 [1610999774434109/end/3 (pid 36913)] 01-18 11:56:19 doltpy.core.system_helpers INFO     Before exiting cleaning up child processes
2021-01-18 11:56:19.822 [1610999774434109/end/3 (pid 36913)] 01-18 11:56:19 doltpy.core.system_helpers INFO     No processes to clean up, exiting
2021-01-18 11:56:19.975 [1610999774434109/end/3 (pid 36913)] Task finished successfully.
2021-01-18 11:56:19.976 Done!
01-18 11:56:19 doltpy.core.system_helpers INFO     Before exiting cleaning up child processes
01-18 11:56:19 doltpy.core.system_helpers INFO     No processes to clean up, exiting


In [ ]:
# TODO: pluck flow from last step

In [4]:
!poetry run python3 demo_two.py run --flow-dep MultiFlowDemo1/1610999774434109

Metaflow 2.2.5.post33+gitc20afd4 executing MultiFlowDemo2 for user:max-hoffman
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2021-01-18 11:56:32.769 Workflow starting (run-id 1610999792759585):
2021-01-18 11:56:32.775 [1610999792759585/start/1 (pid 36928)] Task is starting.
2021-01-18 11:56:33.837 [1610999792759585/start/1 (pid 36928)] 01-18 11:56:33 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-18 11:56:33.934 [1610999792759585/start/1 (pid 36928)] 01-18 11:56:33 doltpy.core.dolt INFO     flow_name,run_id,step_name,task_id,kind,database,table_name,commit,timestamp
2021-01-18 11:56:33.939 [1610999792759585/start/1 (pid 36928)] MultiFlowDemo1,1610999774434109,start,1,read,.,bar,m7t6n1cabcmghfqav3jsti66mpcqs4go,1.6109998e+09
2021-01-18 11:56:33.940 [1610999792759585/start/1 (pid 36928)] 
2021-01-18 11:56:33.940 [1610999792759585/start/1 (pid 36928)] 01-18 11:56:33 doltpy.core.dolt INFO  

2021-01-18 11:56:36.167 [1610999792759585/middle/2 (pid 36980)] 01-18 11:56:36 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-18 11:56:36.195 [1610999792759585/middle/2 (pid 36980)] 01-18 11:56:36 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-18 11:56:36.254 [1610999792759585/middle/2 (pid 36980)] 01-18 11:56:36 doltpy.core.dolt INFO     * master                                        	a7hoh2l3phq5irslbupo7qpljd936kk5
2021-01-18 11:56:36.364 [1610999792759585/middle/2 (pid 36980)] 
2021-01-18 11:56:36.364 [1610999792759585/middle/2 (pid 36980)] 01-18 11:56:36 doltpy.core.write.write INFO     No import mode specified, table exists, using "update"
2021-01-18 11:56:36.453 [1610999792759585/middle/2 (pid 36980)] 01-18 11:56:36 doltpy.core.write.write INFO     Importing to table baz in dolt directory located in ., import mode update
2021-01-18 11:56:36.454 [1610999792759585/m

In [6]:
!cat demo_two.py

import logging

logger = logging.getLogger()
logger.setLevel(logging.WARNING)

import pickle
import time

from metaflow import FlowSpec, step, DoltDT, Parameter
from metaflow.datatools.dolt import DoltRun
import pandas as pd
from sklearn import tree

class MultiFlowDemo2(FlowSpec):

    flow_dep = Parameter('flow-dep',  help="Specifc the tag for the input version", required=True)

    @step
    def start(self):
        flow, run = self.flow_dep.split("/")
        d = DoltRun(flow_name=flow, run_id=run)
        f_input = d.reads[0]
        f_output = d.writes[0]
        with DoltDT(run=self) as dolt:
            self.inp1 = dolt.read_table(f_input.table_name, commit=f_input.commit)
            self.inp2 = dolt.read_table(f_output.table_name, commit=f_output.commit)

        self.next(self.middle)

    @step
    def middle(self):
        with DoltDT(run=self) as dolt:

            df = self.inp1 + self.inp2



        self.next(self.end)

    @step
    def end(self):
        pass


if __